In [2]:
import numpy as np
import pandas as pd
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
import yellowbrick as yb
import folium
sns.set(style="whitegrid")
pd.set_option('display.max_columns',None)

In [3]:
df_values = pd.read_csv('Data/values.csv')
y = pd.read_csv('Data/labels.csv')

In [ ]:
X.head(20)

In [4]:
df_values.columns

Index(['id', 'amount_tsh', 'date_recorded', 'funder', 'gps_height',
       'installer', 'longitude', 'latitude', 'wpt_name', 'num_private',
       'basin', 'subvillage', 'region', 'region_code', 'district_code', 'lga',
       'ward', 'population', 'public_meeting', 'recorded_by',
       'scheme_management', 'scheme_name', 'permit', 'construction_year',
       'extraction_type', 'extraction_type_group', 'extraction_type_class',
       'management', 'management_group', 'payment', 'payment_type',
       'water_quality', 'quality_group', 'quantity', 'quantity_group',
       'source', 'source_type', 'source_class', 'waterpoint_type',
       'waterpoint_type_group'],
      dtype='object')

In [37]:
# Need to drop features that have no influence, and are repetitive
X = df_values.drop(columns=['id', 'amount_tsh', 'date_recorded', 'funder',
       'longitude', 'latitude', 'wpt_name', 'num_private',
       'subvillage', 'region', 'region_code', 'district_code',
       'ward', 'public_meeting', 'recorded_by',
       'scheme_management', 'scheme_name', 'permit', 
       'extraction_type_group', 'extraction_type_class',
       'management_group', 'payment', 'payment_type',
       'quality_group', 'quantity', 'quantity_group',
       'source_type', 'source_class',
       'waterpoint_type_group'])

In [38]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   gps_height         59400 non-null  int64 
 1   installer          55745 non-null  object
 2   basin              59400 non-null  object
 3   lga                59400 non-null  object
 4   population         59400 non-null  int64 
 5   construction_year  59400 non-null  int64 
 6   extraction_type    59400 non-null  object
 7   management         59400 non-null  object
 8   water_quality      59400 non-null  object
 9   source             59400 non-null  object
 10  waterpoint_type    59400 non-null  object
dtypes: int64(3), object(8)
memory usage: 5.0+ MB


In [31]:
X.water_quality.unique()

array(['soft', 'salty', 'milky', 'unknown', 'fluoride', 'coloured',
       'salty abandoned', 'fluoride abandoned'], dtype=object)

In [32]:
X.isna().sum().sum()

3655

In [39]:
X.dropna(inplace=True)
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55745 entries, 0 to 59399
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   gps_height         55745 non-null  int64 
 1   installer          55745 non-null  object
 2   basin              55745 non-null  object
 3   lga                55745 non-null  object
 4   population         55745 non-null  int64 
 5   construction_year  55745 non-null  int64 
 6   extraction_type    55745 non-null  object
 7   management         55745 non-null  object
 8   water_quality      55745 non-null  object
 9   source             55745 non-null  object
 10  waterpoint_type    55745 non-null  object
dtypes: int64(3), object(8)
memory usage: 5.1+ MB


In [42]:
X.duplicated()

0        False
1        False
2        False
3        False
4        False
         ...  
59394    False
59395    False
59396    False
59398     True
59399    False
Length: 55745, dtype: bool

In [35]:
y['id'] = y.id.astype(int)

In [ ]:
y.head()

In [43]:
df_all = df_values.join(y.set_index('id'), on='id')

In [44]:
df_all.head(30)

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322e+00,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466e+00,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329e+00,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-1.115530e+01,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359e+00,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
5,9944,20.0,2011-03-13,Mkinga Distric Coun,0,DWE,39.172796,-4.765587e+00,Tajiri,0,Pangani,Moa/Mwereme,Tanga,4,8,Mkinga,Moa,1,True,GeoData Consultants Ltd,VWC,Zingibali,True,2009,submersible,submersible,submersible,vwc,user-group,pay per bucket,per bucket,salty,salty,enough,enough,other,other,unknown,communal standpipe multiple,communal standpipe,functional
6,19816,0.0,2012-10-01,Dwsp,0,DWSP,33.362410,-3.766365e+00,Kwa Ngomho,0,Internal,Ishinabulandi,Shinyanga,17,3,Shinyanga Rural,Samuye,0,True,GeoData Consultants Ltd,VWC,NaN,True,0,swn 80,swn 80,handpump,vwc,user-group,never pay,never pay,soft,good,enough,enough,machine dbh,borehole,groundwater,hand pump,hand pump,non functional
7,54551,0.0,2012-10-09,Rwssp,0,DWE,32.620617,-4.226198e+00,Tushirikiane,0,Lake Tanganyika,Nyawishi Center,Shinyanga,17,3,Kahama,Chambo,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,nira/tanira,nira/tanira,handpump,wug,user-group,unknown,unknown,milky,milky,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump,non functional
8,53934,0.0,2012-11-03,Wateraid,0,Water Aid,32.711100,-5.146712e+00,Kwa Ramadhan Musa,0,Lake Tanganyika,Imalauduki,Tabora,14,6,Tabora Urban,Itetemia,0,True,GeoData Consultants Ltd,VWC,NaN,True,0,india mark ii,india mark ii,handpump,vwc,user-group,never pay,never pay,salty,salty,seasonal,seasonal,machine dbh,borehole,groundwater,hand pump,hand pump,non functional
9,46144,0.0,2011-08-03,Isingiro Ho,0,Artisan,30.626991,-1.257051e+00,Kwapeto,0,Lake Victoria,Mkonomre,Kagera,18,1,Karagwe,Kaisho,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,nira/tanira,nira/tanira,handpump,vwc,user-group,never pay,never pay,soft,good,enough,enough,shallo

In [50]:
df_all.duplicated(subset='id').value_counts()

False    59400
dtype: int64

In [48]:
df_all.drop_duplicates(inplace=True)

In [49]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59400 entries, 0 to 59399
Data columns (total 41 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     59400 non-null  int64  
 1   amount_tsh             59400 non-null  float64
 2   date_recorded          59400 non-null  object 
 3   funder                 55765 non-null  object 
 4   gps_height             59400 non-null  int64  
 5   installer              55745 non-null  object 
 6   longitude              59400 non-null  float64
 7   latitude               59400 non-null  float64
 8   wpt_name               59400 non-null  object 
 9   num_private            59400 non-null  int64  
 10  basin                  59400 non-null  object 
 11  subvillage             59029 non-null  object 
 12  region                 59400 non-null  object 
 13  region_code            59400 non-null  int64  
 14  district_code          59400 non-null  int64  
 15  lg

In [51]:
df_all_no_id = df_all.drop('id', axis=1)

In [52]:
df_all_no_id.duplicated().value_counts()

False    59364
True        36
dtype: int64

In [ ]:
X.management.unique()

In [ ]:
df_lat_long = X['latitude'].to_frame().join(X['longitude']).join(y)

In [ ]:
df_lat_long.set_index('id')

In [ ]:
map_center = [df_lat_long['latitude'].mean(), df_lat_long['longitude'].mean()]

In [ ]:
map1 = folium.Map(location = map_center, tiles='Openstreetmap', zoom_start = 5, control_scale=True, prefer_canvas=True)
for index, loc in df_lat_long.iterrows():
    if loc['status_group']==0:
        color = 'green'
    elif loc['status_group']==1:
        color = "red"
    elif loc['status_group']==2:
        color = 'blue'
    else:
        color = 'black'
    folium.CircleMarker([loc['latitude'], loc['longitude']], radius=2, weight=1, popup=loc['id'], fill_color=color, stroke=False).add_to(map1)
# folium.vector_layers.path_options(stroke=False)
folium.LayerControl().add_to(map1)

map1

In [ ]:
X.district.value_counts()

In [ ]:
y.head()

In [ ]:
y.status_group.unique()

In [ ]:
y = y.replace({'status_group': {'functional' : 0, 
                                'non functional' : 1, 
                                'functional needs repair' : 2}})

In [ ]:
y.status_group.value_counts()